In [30]:
'''
Access AO integrals

Mole.intor and Mole.intor_by_shell functions can generate AO integrals.
Calling Mole.intor with the integral function name returns a integral matrix
for all basis functions defined in Mole.  If the integral operator has many
compenents eg gradients.

Mole.intor_by_shell function generates the integrals for the given shell
indices.

See pyscf/gto/moleintor.py file for the complete list of supported integrals.
'''


'\nAccess AO integrals\n\nMole.intor and Mole.intor_by_shell functions can generate AO integrals.\nCalling Mole.intor with the integral function name returns a integral matrix\nfor all basis functions defined in Mole.  If the integral operator has many\ncompenents eg gradients.\n\nMole.intor_by_shell function generates the integrals for the given shell\nindices.\n\nSee pyscf/gto/moleintor.py file for the complete list of supported integrals.\n'

In [31]:
import numpy
from pyscf import gto, scf

mol = gto.M(
    verbose = 0,
    atom = 'C 0 0 0; O 0 0 1.5',
    basis = 'ccpvdz'
)
mf = scf.RHF(mol)
mf.kernel()
dm = mf.make_rdm1()

#

In [32]:
# Overlap, kinetic, nuclear attraction
#
s = mol.intor('int1e_ovlp')
t = mol.intor('int1e_kin')
v = mol.intor('int1e_nuc')
# Overlap, kinetic, nuclear attraction gradients (against electron coordinates
# on bra)
s1 = mol.intor('int1e_ipovlp')  # (3,N,N) array, 3 for x,y,z components
t1 = mol.intor('int1e_ipkin' )  # (3,N,N) array, 3 for x,y,z components
v1 = mol.intor('int1e_ipnuc' )  # (3,N,N) array, 3 for x,y,z components

mol.set_common_origin([0,0,0])  # Set gauge origin before computing dipole integrals
print('Dipole %s' % numpy.einsum('xij,ji->x', mol.intor('int1e_r'), dm))

#

Dipole [ 1.76281364e-15 -7.23066046e-16  2.34920986e+01]


In [33]:
# AO overlap between two molecules
#
mol1 = gto.M(
    verbose = 0,
    atom = 'H 0 1 0; H 1 0 0',
    basis = 'ccpvdz'
)
s = gto.intor_cross('int1e_ovlp', mol, mol1)
print('overlap shape (%d, %d)' % s.shape)


overlap shape (28, 10)


In [34]:
#
# 2e integrals.  Keyword aosym is to specify the permutation symmetry in the
# AO integral matrix.  s8 means 8-fold symmetry, s2kl means 2-fold symmetry
# for the symmetry between kl in (ij|kl)
#
eri = mol.intor('int2e', aosym='s8')


#
# 2e gradient integrals (against electronic coordinates) on bra of first atom.
# aosym=s2kl indicates that the permutation symmetry is used on k,l indices of
# (ij|kl). The resultant eri is a 3-dimension array (3, N*N, N*(N+1)/2) where
# N is the number of AO orbitals.
#
eri = mol.intor('int2e_ip1', aosym='s2kl')


In [35]:
#
# Settting aosym=s1 (the default flag) leads to a 3-dimension (3, N*N, N*N)
# eri array.
#
nao = mol.nao_nr()
eri = mol.intor('int2e_ip1').reshape(3,nao,nao,nao,nao)


#

In [36]:
atm_id = 1  # second atom
bas_start, bas_end, ao_start, ao_end = mol.aoslice_by_atom()[atm_id]
tot_bra = ao_end - ao_start
nao = mol.nao_nr()
eri1 = numpy.empty((3,tot_bra,nao,nao,nao))
pi = 0
for i in range(mol.nbas):
    if mol.bas_atom(i) == atm_id:
        pj = 0
        for j in range(mol.nbas):
            pk = 0
            for k in range(mol.nbas):
                pl = 0
                for l in range(mol.nbas):
                    shls = (i, j, k, l)
                    buf = mol.intor_by_shell('int2e_ip1_sph', shls)
                    comp_3, di, dj, dk, dl = buf.shape
                    eri1[:,pi:pi+di,pj:pj+dj,pk:pk+dk,pl:pl+dl] = buf
                    pl += dl
                pk += dk
            pj += dj
        pi += di
print('integral shape %s' % str(eri1.shape))
# This integral block can be generated using mol.intor
eri1 = mol.intor('int2e_ip1_sph', shls_slice=(bas_start, bas_end,
                                              0, mol.nbas,
                                              0, mol.nbas,
                                              0, mol.nbas))


#

integral shape (3, 14, 28, 28, 28)


In [37]:
# Generate a sub-block of AO integrals.  The sub-block (ij|kl) contains the
# shells 2:5 for basis i, 0:2 for j, 0:4 for k and 1:3 for l
#

In [38]:
sub_eri = mol.intor('int2e', shls_slice=(2,5,0,2,0,4,1,3))

# This statement is equivalent to
dims = []
for i in range(mol.nbas):
    l = mol.bas_angular(i)
    nc = mol.bas_nctr(i)
    dims.append((l * 2 + 1) * nc)
nao_i = sum(dims[2:5])
nao_j = sum(dims[0:2])
nao_k = sum(dims[0:4])
nao_l = sum(dims[1:3])
sub_eri = numpy.empty((nao_i,nao_j,nao_k,nao_l))
pi = 0
for i in range(2,5):
    pj = 0
    for j in range(0,2):
        pk = 0
        for k in range(0,4):
            pl = 0
            for l in range(1,3):
                shls = (i, j, k, l)
                buf = mol.intor_by_shell('int2e_sph', shls)
                di, dj, dk, dl = buf.shape
                sub_eri[pi:pi+di,pj:pj+dj,pk:pk+dk,pl:pl+dl] = buf
                pl += dl
            pk += dk
        pj += dj
    pi += di
sub_eri = sub_eri.reshape(nao_i*nao_j,nao_k*nao_l)

#

In [39]:
# 2-electron integrals over different molecules. E.g. a,c of (ab|cd) on one molecule
# and b on another molecule and d on the third molecule.
#
mol1 = mol
mol2 = gto.M(atom='He', basis='ccpvdz')
mol3 = gto.M(atom='O', basis='sto-3g')

mol123 = mol1 + mol2 + mol3
eri = mol123.intor('int2e', shls_slice=(0, mol1.nbas,
                                        mol1.nbas, mol1.nbas+mol2.nbas,
                                        0, mol1.nbas,
                                        mol1.nbas+mol2.nbas, mol123.nbas))


#
# Generate all AO integrals for a sub-system.
#

In [40]:
mol = gto.M(atom=[['H', 0,0,i] for i in range(10)])
atom_idx = [0,2,4]  # atoms in the sub-system
sub_mol = mol.copy()
sub_mol._bas = mol._bas[atom_idx]
sub_eri = sub_mol.intor('int2e', aosym='s1')

# This statement is equivalent to
sub_nao = 0
for i in range(mol.nbas):
    if mol.bas_atom(i) in atom_idx:
        l = mol.bas_angular(i)
        nc = mol.bas_nctr(i)
        sub_nao += (l * 2 + 1) * nc
sub_eri = numpy.empty((sub_nao,sub_nao,sub_nao,sub_nao))
pi = 0
for i in range(mol.nbas):
    if mol.bas_atom(i) in atom_idx:
        pj = 0
        for j in range(mol.nbas):
            if mol.bas_atom(j) in atom_idx:
                pk = 0
                for k in range(mol.nbas):
                    if mol.bas_atom(k) in atom_idx:
                        pl = 0
                        for l in range(mol.nbas):
                            if mol.bas_atom(l) in atom_idx:
                                shls = (i, j, k, l)
                                buf = mol.intor_by_shell('int2e_sph', shls)
                                di, dj, dk, dl = buf.shape
                                sub_eri[pi:pi+di,pj:pj+dj,pk:pk+dk,pl:pl+dl] = buf
                                pl += dl
                        pk += dk
                pj += dj
        pi += di
sub_eri = sub_eri.reshape(sub_nao**2,sub_nao**2)



In [41]:
'''
AO integrals from spherical GTO basis representation to spinor-GTO basis
representation.

Generally, the transformation requires two steps.  First is to form a
quaternion matrix (2x2 super matrix) using Pauli matrices (sigma_2x2)

        1_2x2 + 1j*sigma_2x2

Second is to contract to the Clebsch-Gordan coefficients (spherical to spinor
transformation coefficients).
'''

import numpy
from pyscf import gto, lib

mol = gto.M(atom = 'H 0 0 0 \n F 0 0 1.1',
            basis = 'cc-pvdz')

nao = mol.nao_nr()

paulix, pauliy, pauliz = lib.PauliMatrices
iden = numpy.eye(2)


In [42]:
################################################################################
#
# Integrals without pauli matrices.
#
################################################################################
ints_spinor = mol.intor('int1e_nuc_spinor')

ints_sph = mol.intor('int1e_nuc_sph')
ints_sph = numpy.einsum('ij,pq->ijpq', iden, ints_sph)

c = mol.sph2spinor_coeff()

ints_from_sph = numpy.einsum('ipa,ijpq,jqb->ab', numpy.conj(c), ints_sph, c)

print(abs(ints_from_sph - ints_spinor).max())

################################################################################
#
# Integrals with pauli matrices and they are real in the spherical GTO basis
# representation
#

3.552713678800501e-15


In [43]:
################################################################################
ints_spinor = mol.intor('int1e_spnucsp_spinor')

# When integrals contains Pauli matrices, they spherical representation have
# four components. The first three correspond to the three Pauli matrices and
# the last one corresponds to identity of quaternion.
ints_sph = mol.intor('int1e_spnucsp_sph', comp=4)
ints_sx = ints_sph[0]
ints_sy = ints_sph[1]
ints_sz = ints_sph[2]
ints_1 = ints_sph[3]
ints_sph = (numpy.einsum('ij,pq->ijpq', 1j*paulix, ints_sx) +
            numpy.einsum('ij,pq->ijpq', 1j*pauliy, ints_sy) +
            numpy.einsum('ij,pq->ijpq', 1j*pauliz, ints_sz) +
            numpy.einsum('ij,pq->ijpq', iden     , ints_1 ))

c = mol.sph2spinor_coeff()

ints_from_sph = numpy.einsum('ipa,ijpq,jqb->ab', numpy.conj(c), ints_sph, c)
print(abs(ints_from_sph - ints_spinor).max())


1.1368683772161603e-13


In [44]:
################################################################################
#
# Integrals with pauli matrices and they are pure imaginary numbers in the
# spherical GTO basis representation
#
################################################################################
ints_spinor = mol.intor('int1e_cg_sa10nucsp_spinor', comp=3)

ints_sph = mol.intor('int1e_cg_sa10nucsp_sph', comp=12).reshape(3,4,nao,nao)
ints_sx = ints_sph[:,0]
ints_sy = ints_sph[:,1]
ints_sz = ints_sph[:,2]
ints_1 = ints_sph[:,3]
# In the integral <r \times \sigma V \sigma \dot p >, the coefficients of the
# quaternion basis are pure imaginary numbers.  The integral engine returns
# the imaginary part of the coefficients (thus multiplying them by factor 1j).
ints_sph = 1j * (numpy.einsum('ij,xpq->xijpq', 1j*paulix, ints_sx) +
                 numpy.einsum('ij,xpq->xijpq', 1j*pauliy, ints_sy) +
                 numpy.einsum('ij,xpq->xijpq', 1j*pauliz, ints_sz) +
                 numpy.einsum('ij,xpq->xijpq', iden     , ints_1 ))

c = mol.sph2spinor_coeff()

ints_from_sph = numpy.einsum('ipa,xijpq,jqb->xab', numpy.conj(c), ints_sph, c)

print(abs(ints_from_sph - ints_spinor).max())

#

5.684341886080802e-14


In [45]:
#
# Integrals (LS|LS) related to Gaunt term
#
# Note the order of spin operators
# SIGMA1X * SIGMA2X     0
# SIGMA1Y * SIGMA2X     1
# SIGMA1Z * SIGMA2X     2
# I1_2x2  * SIGMA2X     3
# SIGMA1X * SIGMA2Y     4
# SIGMA1Y * SIGMA2Y     5
# SIGMA1Z * SIGMA2Y     6
# I1_2x2  * SIGMA2Y     7
# SIGMA1X * SIGMA2Z     8
# SIGMA1Y * SIGMA2Z     9
# SIGMA1Z * SIGMA2Z     10
# I1_2x2  * SIGMA2Z     11
# SIGMA1X * I2_2x2      12
# SIGMA1Y * I2_2x2      13
# SIGMA1Z * I2_2x2      14
# I1_2x2  * I2_2x2      15
gaunt_spinor = mol.intor('int2e_ssp1ssp2_spinor')
gaunt_sph = mol.intor('int2e_ssp1ssp2')
si = numpy.array([paulix * 1j, pauliy * 1j, pauliz * 1j, iden])
# Be careful with the order of the 16 components:
# index for electron 1 runs inside, index for electron 2 runs outside
ints_from_sph = lib.einsum('xypqrs,xij,ykl,ipa,jqb,krc,lsd->abcd',
                           gaunt_sph.reshape(4,4,nao,nao,nao,nao).transpose(1,0,2,3,4,5),
                           si, si, c.conj(), c, c.conj(), c)
print(abs(ints_from_sph - gaunt_spinor).max())

#
# (SS|SS) for four small component basis functions in Dirac-Coulomb interaction
#
ssss_spinor = mol.intor('int2e_spsp1spsp2_spinor')
ssss_sph = mol.intor('int2e_spsp1spsp2')
si = numpy.array([paulix * 1j, pauliy * 1j, pauliz * 1j, iden])
# Be careful with the order of the 16 components:
# index for electron 1 runs inside, index for electron 2 runs outside
ssss_from_sph = lib.einsum('xypqrs,xij,ykl,ipa,jqb,krc,lsd->abcd',
                           ssss_sph.reshape(4,4,nao,nao,nao,nao).transpose(1,0,2,3,4,5),
                           si, si, c.conj(), c, c.conj(), c)
print(abs(ssss_from_sph - ssss_spinor).max())


1.4210854715202004e-14
1.3642420526593924e-12
